In [1]:
from huggingface_hub import notebook_login
notebook_login()

In [2]:
import json
import pandas as pd 
import torch
from datasets import Dataset, load_dataset
from peft import LoraConfig, PeftModel
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    TrainingArguments,
    pipeline,
    logging
)
from trl import SFTTrainer

/home/dlvaayuai/anaconda3/envs/deep_lrnr/lib/python3.9/site-packages/torch/cuda/__init__.py:611: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
2024-03-04 18:30:14.464969: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-04 18:30:14.465008: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-04 18:30:14.465988: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-04 18:30:14.470723: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To en

In [3]:
model_id = "google/gemma-7b-it"

In [4]:
model = AutoModelForCausalLM.from_pretrained(model_id, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(model_id, add_eos_token=True)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
{
    "instruction": "Create a function to calculate the sum of a sequence of integers.",
    "input":"[1, 2, 3, 4, 5]",
    "output": "# Python code def sum_sequence(sequence): sum = 0 for num in sequence: sum += num return sum"
}

In [ ]:
{
    "question":"How many heads of the departments are older than 56 ?"
    "context":"CREATE TABLE head (age INTEGER)"
    "answer":"SELECT COUNT(*) FROM head WHERE age > 56"
}

In [5]:
dataset = load_dataset("b-mc2/sql-create-context", split="train")

Using the latest cached version of the dataset since b-mc2/sql-create-context couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'default' at /home/dlvaayuai/.cache/huggingface/datasets/b-mc2___sql-create-context/default/0.0.0/9d80a6a118b838d9defc3798d659a54a2ac2ff37 (last modified on Mon Mar  4 18:24:52 2024).


In [8]:
type(dataset)

datasets.arrow_dataset.Dataset

In [9]:
dataset[0]

{'context': 'CREATE TABLE head (age INTEGER)',
 'answer': 'SELECT COUNT(*) FROM head WHERE age > 56',
 'question': 'How many heads of the departments are older than 56 ?'}

## **GEMMA CHAT FORMAT**

```
<bos><start_of_turn>user
Write a hello world program<end_of_turn>
<start_of_turn>model
```

###**this is a really important step.**

** Hence Input should be as below**
```
Dataset sample :-
    {
    "question":"How many heads of the departments are older than 56 ?"
    "context":"CREATE TABLE head (age INTEGER)"
    "answer":"SELECT COUNT(*) FROM head WHERE age > 56"
}

Input format:-

<start_of_turn>user
Below is an instruction that describes a task. Write a response that ' \\
               'appropriately completes the request.\\n\\n
How many heads of the departments are older than 56 ?
here is the context CREATE TABLE head (age INTEGER)<end_of_turn>\\n
<start_of_turn>model
 SELECT COUNT(*) FROM head WHERE age > 56<end_of_turn>

 ```

In [6]:
def generate_prompt(data_point):
    """
    Generate prompt format compatible for the LLM model using Quation, Context and answer 
    :param Data_point: Dict: Data point
    :return: Dict:tokenized prompt
    """
    prefix_text = 'Below is an Question with Context that describes a SQL task. Write a response that ' \
        'appropriately completes the request.\\n\\n'
        
    text = f"""<start_of_turn>user {prefix_text} {data_point["question"]} here is the context {data_point["context"]} <end_of_turn>\\n<start_of_turn>model{data_point["answer"]} <end_of_turn>"""
    
    return text

In [7]:
text_column = [generate_prompt(data_point) for data_point in dataset]

dataset = dataset.add_column("prompt", text_column)

In [12]:
dataset[0]

{'context': 'CREATE TABLE head (age INTEGER)',
 'answer': 'SELECT COUNT(*) FROM head WHERE age > 56',
 'question': 'How many heads of the departments are older than 56 ?',
 'prompt': '<start_of_turn>user Below is an Question with Context that describes a SQL task. Write a response that appropriately completes the request.\\n\\n How many heads of the departments are older than 56 ? here is the context CREATE TABLE head (age INTEGER) <end_of_turn>\\n<start_of_turn>modelSELECT COUNT(*) FROM head WHERE age > 56 <end_of_turn>'}

In [8]:
dataset = dataset.shuffle(seed=2024)
dataset = dataset.map(lambda samples: tokenizer(samples["prompt"]), batched=True)

In [9]:
dataset = dataset.train_test_split(test_size=0.2)
train_data = dataset["train"]
test_data = dataset["test"]

In [10]:
from peft import LoraConfig, get_peft_model
lora_config = LoraConfig(
    r = 256,
    lora_alpha = 128,
    target_modules=['o_proj', 'q_proj', 'up_proj', 'v_proj', 'k_proj', 'down_proj', 'gate_proj'],
    lora_dropout=0.05,
    bias = "none",
    task_type = "CAUSAL_LM"
)

In [11]:
model = get_peft_model(model, lora_config)

In [12]:
trainable, total = model.get_nb_trainable_parameters()
print(f"Trainable:{trainable} | total: {total} | percentage: {(trainable/total)*100:.4f}%")

Trainable:800063488 | total: 9337744384 | percentage: 8.5681%


In [13]:
import transformers

from trl import SFTTrainer

tokenizer.pad_token = tokenizer.eos_token
torch.cuda.empty_cache()
trainer = SFTTrainer(
    model = model,
    train_dataset = train_data,
    eval_dataset = test_data,
    dataset_text_field = "prompt",
    peft_config = lora_config,
    args = transformers.TrainingArguments(
        per_device_train_batch_size = 1,
        gradient_accumulation_steps = 4,
        warmup_steps = 0.03,
        max_steps = 100,
        learning_rate = 2e-4,
        logging_steps = 1,
        output_dir = "outputs",
        optim = "paged_adamw_32bit",
        save_strategy = "epoch"
    ),
    data_collator = transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)
) 

/home/dlvaayuai/anaconda3/envs/deep_lrnr/lib/python3.9/site-packages/trl/trainer/sft_trainer.py:223: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/62861 [00:00<?, ? examples/s]

Map:   0%|          | 0/15716 [00:00<?, ? examples/s]

/home/dlvaayuai/anaconda3/envs/deep_lrnr/lib/python3.9/site-packages/trl/trainer/sft_trainer.py:290: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(


In [14]:
trainer.train()

new_model = "gemma-it-sql-finetune-test"
trainer.model.save_pretrained(new_model)

Initialized DVC repository.

You can now commit the changes to git.



SCMError: Empty git repo